## Описание Use Case:
Имеется система, состоящая из 1 млн. элементов (например сервера) по каждому элементу у нас задано 4 KPI:
- Среднее значение метрик за последние 20 отсчетов
- Медианное значение метрик за весь период
- Количество не нулевых метрик
- Количество метрик превышающих среднее по всем элементам

Всистему постоянно поступают данные (Например статистика по сиполняемым на серверах приложениям) по закону пуассоновского потока (не известно в какой момент времени придут данные и по какому элементу)

Данные требуется хранить в детальном виде, все метрики необхоимо расчитывать "на лету" при запросе

In [ ]:
# Создадим Key-Value (HashTable) storage на 1 млн объектов
# Ключем будет являтс id объекта от 0 до 1.000.000, значением будет являтся пустой массив
kv_storage = {i:[] for i in range(1000000)}
len(kv_storage)

1000000

In [ ]:
# На текущий момент наше хранилище пусто
kv_storage[0]

[]

In [ ]:
# Заполним наше ПУСТОЕ хранилище для каждого ключа произвольным количеством событий со случайными знаяениями:
import numpy as np
for k in kv_storage.keys():
  num_events = np.random.randint(0,320)
  new_val = np.random.uniform(low=0,high=50,size=num_events).tolist()
  kv_storage[k].extend(new_val)

In [ ]:
# Теперь у нас есть данные вхранилище
kv_storage[0]

[5.740670599935848,
 10.948260676297068,
 27.492018566407957,
 49.18444902183643,
 40.94120348257702,
 26.580212065682023,
 34.79536982744799,
 40.08304006560846,
 20.67082168430561,
 8.13169382721924,
 24.219171974493108,
 45.87152193456262,
 33.243416089711985,
 4.091091498170263,
 5.724553362442619,
 35.699402946309434,
 18.75135571378249,
 35.69992715840923,
 37.19044780956082,
 46.21089004219686,
 48.68260442800144,
 25.0494713405596,
 48.42455602827356,
 12.996752230310882,
 44.40272724014333,
 40.86227363066401,
 23.706022956393753,
 47.205897823629186,
 9.890046014112286,
 6.007544395871401,
 33.558863436370096,
 1.764040269907724,
 21.35281133438979,
 27.81420999466768,
 48.258078315522965,
 3.887024678679285,
 30.853542799174537,
 29.592385584878144,
 8.719283698825453,
 16.228077527149715,
 13.886891821746245,
 35.3970270106454,
 9.466372066197831,
 30.750983750309846,
 8.936835720496733,
 4.106970043209962,
 34.21507305423267,
 1.6618591980099573,
 1.7109089540813638,
 33.5

In [ ]:
#Проверим скорость доступа к случайному ключу
%%timeit
kv_storage[np.random.randint(0,1000000)]

The slowest run took 242.71 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 1.39 µs per loop


In [ ]:
#Теперь напишем простенький класс для расчета KPI
class KpiBuilder():
  def __init__(self, key):
        self.key = key

  def get(self):
    return kv_storage.get(self.key)

  def mean_20(self):
    return sum(self.get()[:-20])/20.

  def median(self):
    m_ = self.get()
    m_.sort()
    if len(m_)%2==1:
      return m_[(len(m_)/2)]
    else:
      return sum(m_[(len(m_)/2)-1:(len(m_)/2)+1])/2.

  def non_zero_cnt(self):
    v = self.get()
    return len([x for x in v if x > 0])

  def cnt_above_mean(self):
    v = self.get()
    m_ = kv_storage['sum_el']/ float(kv_storage['len_el'])

    print ("Mean across all elements: ",m_)
    return len([x for x in v if x > m_])




# Количество метрик превышающих среднее по всем элементам


In [ ]:
# Инициализируем вспомогательное значение sum_el/ len_el суммой элементов
v_len = len([item for sublist in kv_storage.values() for item in sublist])
v_sum = sum([item for sublist in kv_storage.values() for item in sublist])

kv_storage['len_el']=v_len
kv_storage['sum_el']=v_sum


In [ ]:
key_123 = KpiBuilder(123)

In [ ]:
key_123.mean_20()

1.2511955801978718

In [ ]:
key_123.median()

12.511955801978718

In [ ]:
key_123.non_zero_cnt()

2

In [ ]:
key_123.cnt_above_mean()

('Mean across all elements: ', 25.000036117437052)


0

In [ ]:
key_123.get()

[1.2612581564067582, 23.76265344755068, 13.405550621391393]

In [ ]:
# Функция которая будет добавлять новые данные в key-value storage
keys = [123]
for k in keys:
  num_events = np.random.randint(0,5)
  new_val = np.random.uniform(low=0,high=50,size=num_events).tolist()
  kv_storage[k].extend(new_val)
  

In [ ]:
key_123.mean_20()

10.526204239451621

In [ ]:
key_123.median()

22.10742330171295

In [ ]:
key_123.non_zero_cnt()

9

In [ ]:
key_123.cnt_above_mean()

('Mean across all elements: ', 25.00003618324138)


3

ДЗ Лайт

Задание 1
Используя код с занятия создайте key-value storage на 1 миллион ключей, заполните его случайными значениями. Придумайте 3 любые метрики по аналогии с метриками с урока, допишите в класс **KpiBuilder** функции, которые будут считать эти метрики, продемонстрируйте расчет. 

Задание 2
Используя функцию с занятия: # Функция которая будет добавлять новые данные в key-value storage
```
keys = [123]
for k in keys:
  num_events = np.random.randint(0,5)
  new_val = np.random.uniform(low=0,high=50,size=num_events).tolist()
  kv_storage[k].extend(new_val)
```
Добавьте новые значения в key-value хранилище из Задания 1 для 1000 случайных ключей (из миллиона)
Возьмите любой ключ, по которому были добавлены данные и выведите расчет метрик из Задания 1.

Задание 3
Модифицируйте функцию с занятия: # Функция которая будет добавлять новые данные в key-value storage

```
keys = [123]
for k in keys:
  num_events = np.random.randint(0,5)
  new_val = np.random.uniform(low=0,high=50,size=num_events).tolist()
  kv_storage[k].extend(new_val)
```
Таким образом чтобы при добавления любого нового значения ключи:

- kv_storage['len_el'] # Количество всех значений в хранилище
- kv_storage['sum_el'] # Сумма всех значений в хранилище

обновляли сво значения таким образом, чтобы используя эти 2 ключа можно было посчитать среднее значения по всем values в хранилище.
Сравните свой результат с*
- v_len = len([item for sublist in kv_storage.values() for item in sublist])
- v_sum = sum([item for sublist in kv_storage.values() for item in sublist])

*- Постарайтесь придумать альтернативный способ обновления ключей kv_storage['len_el'] и kv_storage['sum_el'] без необходимости проходить по всем значениям в хранилище.


Про

Используя код ниже:
```
import bisect
import hashlib

class ConsistentHash:

  def __init__(self,num_machines=1,num_replicas=1):
    self.num_machines = num_machines
    self.num_replicas = num_replicas
    hash_tuples = [(j,k,my_hash(str(j)+"_"+str(k))) \
                   for j in range(self.num_machines) \
                   for k in range(self.num_replicas)]

    hash_tuples.sort(lambda x,y: cmp(x[2],y[2]))
    self.hash_tuples = hash_tuples

  def get_machine(self,key):

    h = my_hash(key)
    # Закальцовываем хэши
    if h > self.hash_tuples[-1][2]: return self.hash_tuples[0][0]
    hash_values = map(lambda x: x[2],self.hash_tuples)
    index = bisect.bisect_left(hash_values,h)
    return self.hash_tuples[index][0]

def my_hash(key):
  return (int(hashlib.md5(key).hexdigest(),16) % 1000000)/1000000.0
  ```
Напишите функцию которая на вход будет принимать количество машин в кластере , коэфициент репликации и значение ключа, а на выходе сообщать на какой машине находится ключ. Попробуйте заменить функцию из примера на какой-нибудь питоновский пакет со схожими функциями (например: uhashring), протестируйте что получилось, напишите вывод.
